### Merging 2017 Weather Data and Initial Inspection and Cleaning

In [1]:
import os
import pandas as pd
file_list = os.listdir()
flat_file_list = list(filter(lambda x: x.endswith('csv'), file_list)) 
data_frames = []
for file_name in flat_file_list:
    df = pd.read_csv(file_name)
    data_frames.append(df)
merged_frame = pd.concat(data_frames)

In [2]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,11,19.4,11,...,8.0,13.0,53.6,*,35.6,*,0.0,I,999.9,0
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,24,35.0,24,...,12.0,21.0,77.0,*,42.8,*,0.0,I,999.9,0
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,24,52.9,24,...,15.0,21.0,84.2,*,62.6,*,0.0,I,999.9,0
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,22,35.4,22,...,15.0,24.1,75.2,*,50.0,*,0.0,I,999.9,0
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,24,37.9,24,...,2.9,999.9,66.2,*,41.0,*,0.0,I,999.9,0


There appear to be zero and NaN latitude/longitude data.  For now lets verify that we have nonempty entries in these fields.  I can always lookup by station number later if I need to find data near a point.

In [3]:
merged_frame[(merged_frame['LATITUDE'] != 0)].dropna().head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,1001099999,2017-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",23.8,24,16.1,24,...,19.0,34.2,30.9,,22.5,*,0.00,G,999.9,1000
1,1001099999,2017-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.0,24,20.8,24,...,29.1,999.9,30.6,*,20.1,,0.03,G,999.9,1000
2,1001099999,2017-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,24,21.4,24,...,29.7,39.2,30.6,,19.2,,0.05,G,999.9,1000
3,1001099999,2017-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.9,24,18.4,24,...,25.3,36.5,33.3,*,20.5,,0.17,G,999.9,1000
4,1001099999,2017-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.4,24,33.5,24,...,22.3,31.7,37.0,*,28.0,,0.02,G,999.9,10000


There are entries with valid latitude and longitude.  Table will be tidied up and then exported to a merged CSV for DB import.

Temp attributes give the number of measurements averaged for mean temperature.  This will not assist me in modeling, so I am going to drop this column.

In [4]:
merged_frame.drop(merged_frame.columns[7], axis = 1, inplace = True)

In [5]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,DEWP_ATTRIBUTES,SLP,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,11,9999.9,...,8.0,13.0,53.6,*,35.6,*,0.0,I,999.9,0
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,24,9999.9,...,12.0,21.0,77.0,*,42.8,*,0.0,I,999.9,0
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,24,9999.9,...,15.0,21.0,84.2,*,62.6,*,0.0,I,999.9,0
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,22,9999.9,...,15.0,24.1,75.2,*,50.0,*,0.0,I,999.9,0
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,24,9999.9,...,2.9,999.9,66.2,*,41.0,*,0.0,I,999.9,0


The same situation applies with the other attribute fields.  Removing them as well.

In [6]:
merged_frame.drop(['DEWP_ATTRIBUTES', 'MAX_ATTRIBUTES', 'MIN_ATTRIBUTES', 'PRCP_ATTRIBUTES'], axis = 1, inplace=True)

In [7]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,9999.9,0,...,11,3.6,11,8.0,13.0,53.6,35.6,0.0,999.9,0
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,9999.9,0,...,24,4.7,24,12.0,21.0,77.0,42.8,0.0,999.9,0
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,9999.9,0,...,24,5.2,24,15.0,21.0,84.2,62.6,0.0,999.9,0
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,9999.9,0,...,22,8.9,22,15.0,24.1,75.2,50.0,0.0,999.9,0
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,9999.9,0,...,24,0.4,24,2.9,999.9,66.2,41.0,0.0,999.9,0


For Temp, dewpoint, SLP, and STP missing values are reported as 9999.9.  These are going to be replaced with NaN for clarity and DB processing purposes.

In [8]:
import numpy as np

def set_null_9999(arg):
    if arg == 9999.9:
        return np.NaN
    else:
        return arg
    
merged_frame['TEMP'] = merged_frame['TEMP'].apply(set_null_9999) 

In [9]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,9999.9,0,...,11,3.6,11,8.0,13.0,53.6,35.6,0.0,999.9,0
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,9999.9,0,...,24,4.7,24,12.0,21.0,77.0,42.8,0.0,999.9,0
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,9999.9,0,...,24,5.2,24,15.0,21.0,84.2,62.6,0.0,999.9,0
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,9999.9,0,...,22,8.9,22,15.0,24.1,75.2,50.0,0.0,999.9,0
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,9999.9,0,...,24,0.4,24,2.9,999.9,66.2,41.0,0.0,999.9,0


In [10]:
merged_frame['DEWP'] = merged_frame['DEWP'].apply(set_null_9999)
merged_frame['SLP'] = merged_frame['SLP'].apply(set_null_9999) 
merged_frame['STP'] = merged_frame['STP'].apply(set_null_9999)

In [11]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,NaN,0,...,11,3.6,11,8.0,13.0,53.6,35.6,0.0,999.9,0
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,NaN,0,...,24,4.7,24,12.0,21.0,77.0,42.8,0.0,999.9,0
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,NaN,0,...,24,5.2,24,15.0,21.0,84.2,62.6,0.0,999.9,0
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,NaN,0,...,22,8.9,22,15.0,24.1,75.2,50.0,0.0,999.9,0
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,NaN,0,...,24,0.4,24,2.9,999.9,66.2,41.0,0.0,999.9,0


For VISIB, WDSP, MXSPD, and GUST the missing value is encoded as 999.9.  As before replacing with NaN.

In [12]:
def set_null_999(arg):
    if arg == 999.9:
        return np.NaN
    else:
        return arg
    
merged_frame['VISIB'] = merged_frame['VISIB'].apply(set_null_999) 

In [13]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,NaN,0,...,11,3.6,11,8.0,13.0,53.6,35.6,0.0,999.9,0
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,NaN,0,...,24,4.7,24,12.0,21.0,77.0,42.8,0.0,999.9,0
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,NaN,0,...,24,5.2,24,15.0,21.0,84.2,62.6,0.0,999.9,0
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,NaN,0,...,22,8.9,22,15.0,24.1,75.2,50.0,0.0,999.9,0
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,NaN,0,...,24,0.4,24,2.9,999.9,66.2,41.0,0.0,999.9,0


In [14]:
merged_frame['WDSP'] = merged_frame['WDSP'].apply(set_null_999)
merged_frame['MXSPD'] = merged_frame['MXSPD'].apply(set_null_999)
merged_frame['GUST'] = merged_frame['GUST'].apply(set_null_999)

In [15]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,NaN,0,...,11,3.6,11,8.0,13.0,53.6,35.6,0.0,999.9,0
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,NaN,0,...,24,4.7,24,12.0,21.0,77.0,42.8,0.0,999.9,0
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,NaN,0,...,24,5.2,24,15.0,21.0,84.2,62.6,0.0,999.9,0
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,NaN,0,...,22,8.9,22,15.0,24.1,75.2,50.0,0.0,999.9,0
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,NaN,0,...,24,0.4,24,2.9,NaN,66.2,41.0,0.0,999.9,0


In [16]:
merged_frame.drop(['SLP_ATTRIBUTES', 'VISIB_ATTRIBUTES', 'WDSP_ATTRIBUTES'], axis=1, inplace=True)

In [17]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,STP_ATTRIBUTES,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,NaN,999.9,0,6.2,3.6,8.0,13.0,53.6,35.6,0.0,999.9,0
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,NaN,999.9,0,6.2,4.7,12.0,21.0,77.0,42.8,0.0,999.9,0
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,NaN,999.9,0,6.2,5.2,15.0,21.0,84.2,62.6,0.0,999.9,0
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,NaN,999.9,0,6.2,8.9,15.0,24.1,75.2,50.0,0.0,999.9,0
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,NaN,999.9,0,5.3,0.4,2.9,NaN,66.2,41.0,0.0,999.9,0


For snow depth missing data will be reported as 999.9.  However, most stations do not report 0 on days with no snow on ground.  As such I will clean this value to zero on such days.

In [18]:
def set_zero_999(arg):
    if arg == 999.9:
        return 0
    else:
        return arg
    
merged_frame['SNDP'] = merged_frame['SNDP'].apply(set_zero_999)

In [19]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,STP_ATTRIBUTES,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,NaN,999.9,0,6.2,3.6,8.0,13.0,53.6,35.6,0.0,0.0,0
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,NaN,999.9,0,6.2,4.7,12.0,21.0,77.0,42.8,0.0,0.0,0
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,NaN,999.9,0,6.2,5.2,15.0,21.0,84.2,62.6,0.0,0.0,0
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,NaN,999.9,0,6.2,8.9,15.0,24.1,75.2,50.0,0.0,0.0,0
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,NaN,999.9,0,5.3,0.4,2.9,NaN,66.2,41.0,0.0,0.0,0


FRSHTT is a combined indicator for occurences of meteorological conditions.  The interpretation for this field is coppied from the data source readme (https://www.ncei.noaa.gov/data/global-summary-of-the-day/doc/readme.txt).


FRSHTT - Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day of:
                         Fog ('F' - 1st digit).
                         Rain or Drizzle ('R' - 2nd digit).
                         Snow or Ice Pellets ('S' - 3rd digit).
                         Hail ('H' - 4th digit).
                         Thunder ('T' - 5th digit).
                         Tornado or Funnel Cloud ('T' - 6th digit).

In [20]:
def process_FRSHTT(arg):
    string = str(arg)
    if len(string)>6:
        #more than six chars would be invalid
        return np.NaN
    index = 0
    outputs = ['Fog', 'Rain', 'Snow', 'Hail', 'Thunder', 'Tornado']
    output = []
    for char in string:
        if char == '1':
            output.append(outputs[index])
        index+=1
    return "|".join(output)

In [21]:
merged_frame['FRSHTT'] = merged_frame['FRSHTT'].apply(process_FRSHTT)

In [22]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,STP_ATTRIBUTES,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,NaN,999.9,0,6.2,3.6,8.0,13.0,53.6,35.6,0.0,0.0,
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,NaN,999.9,0,6.2,4.7,12.0,21.0,77.0,42.8,0.0,0.0,
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,NaN,999.9,0,6.2,5.2,15.0,21.0,84.2,62.6,0.0,0.0,
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,NaN,999.9,0,6.2,8.9,15.0,24.1,75.2,50.0,0.0,0.0,
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,NaN,999.9,0,5.3,0.4,2.9,NaN,66.2,41.0,0.0,0.0,


In [23]:
merged_frame.drop('STP_ATTRIBUTES', axis=1, inplace=True)

99.99 denotes a missing PRCP value.  Many stations do not report precip if the value is 0.  So I will replace those values with 0.

In [24]:
def set_zero_99(arg):
    if arg == 99.99:
        return 0
    else:
        return arg
    
merged_frame['PRCP'] = merged_frame['PRCP'].apply(set_zero_99)

In [25]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,NaN,999.9,6.2,3.6,8.0,13.0,53.6,35.6,0.0,0.0,
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,NaN,999.9,6.2,4.7,12.0,21.0,77.0,42.8,0.0,0.0,
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,NaN,999.9,6.2,5.2,15.0,21.0,84.2,62.6,0.0,0.0,
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,NaN,999.9,6.2,8.9,15.0,24.1,75.2,50.0,0.0,0.0,
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,NaN,999.9,5.3,0.4,2.9,NaN,66.2,41.0,0.0,0.0,


MAX and MIN also report missing data as 9999.9.  Cleaning as such with NaN.

In [26]:
merged_frame['MAX'] = merged_frame['MAX'].apply(set_null_9999) 
merged_frame['MIN'] = merged_frame['MIN'].apply(set_null_9999)

In [27]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,702699999,2017-02-10,0.0,0.0,7026.0,"WXPOD 7026, AF",46.9,19.4,NaN,999.9,6.2,3.6,8.0,13.0,53.6,35.6,0.0,0.0,
1,702699999,2017-02-11,0.0,0.0,7026.0,"WXPOD 7026, AF",54.4,35.0,NaN,999.9,6.2,4.7,12.0,21.0,77.0,42.8,0.0,0.0,
2,702699999,2017-02-12,0.0,0.0,7026.0,"WXPOD 7026, AF",70.2,52.9,NaN,999.9,6.2,5.2,15.0,21.0,84.2,62.6,0.0,0.0,
3,702699999,2017-02-13,0.0,0.0,7026.0,"WXPOD 7026, AF",61.1,35.4,NaN,999.9,6.2,8.9,15.0,24.1,75.2,50.0,0.0,0.0,
4,702699999,2017-03-16,0.0,0.0,7026.0,"WXPOD 7026, AF",52.7,37.9,NaN,999.9,5.3,0.4,2.9,NaN,66.2,41.0,0.0,0.0,


In [28]:
merged_frame.to_csv('2017-merged-cleaned.csv.dat')#appending .dat to avoid accidental reprocessing if script is rerun.
